In [15]:
import enigma as en
import numpy as np
from importlib import reload

In [17]:
reload(en)

<module 'enigma' from '/home/august/phd/enigma/enigma.py'>

In [18]:
message = en.Message("augustwollter")
# These parameters describe a rotors initial position, 
# and when the next rotor is advanced
pos = 0
notch = ord('a') - 97
rotorV = en.Rotor(np.identity(26), pos, notch)

In [19]:
t, s = rotorV.getTransform()
print(en.matrix_to_word(t))
s
m = np.matmul(np.identity(26), t)
m = np.matmul(t,m)
print(en.matrix_to_word(m))

Next rotor is advanced
abcdefghijklmnopqrstuvwxyz
abcdefghijklmnopqrstuvwxyz


In [26]:
reload(en)
c = en.cyclic_permutation_array('a')
print(c)
type(c)

AttributeError: module 'enigma' has no attribute 'cyclic_permutation_array'

# Cyclic permutatiotns for Rotor I

In [36]:
reload(en)
# Creating Rotor I
t1 = en.cyclic_permutation_matrix('aeltphqxru') 
t2 = en.cyclic_permutation_matrix('bknw') 
t3 = en.cyclic_permutation_matrix('cmoy')
t4 = en.cyclic_permutation_matrix('dfg')
t5 = en.cyclic_permutation_matrix('iv')
t6 = en.cyclic_permutation_matrix('jz')
#s maps to itself
t = np.matmul(t1,t2)
t = np.matmul(t,t3)
t = np.matmul(t,t4)
t = np.matmul(t,t5)
t = np.matmul(t,t6)



26

In [45]:
rotor1 = en.Rotor(t,0,0)
rotor1
#print(rotor1.getTransform())
mess = en.Message('v')
t = rotor1.getTransform()
mess.matrix
#mess.transform(rotor1)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 26)

# Reflector

In [379]:
reload(enigma)
perms = ['ay', 'br', 'cu', 'dh', 'eq', 'fs', 'gl', 
         'ip', 'jx', 'kn', 'mo', 'tz', 'vw']
reflector = en.list_cyclic_permutation_matrix(perms)
print(reflector)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 

In [382]:
print(t)
print(reflector)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 

In [1]:
reload(enigma)
machine = en.Machine()
reflector = en.Reflector(perms)
machine.addTransforms([rotor1, reflector, rotor1])
machine.components[0]

NameError: name 'reload' is not defined

In [406]:
mes_1 = en.Message('augustwollter')

for t in machine.components:
    mes_1.transform(t)
print(mes_1)
for t in machine.components:
    mes_1.transform(t)
print(mes_1)

Next rotor is advanced
Next rotor is advanced
xcqcgvuejjvdm
Next rotor is advanced
Next rotor is advanced
kyzyqhcdpphso
